In [88]:
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Shape
from OCC.Core.TopAbs import TopAbs_SOLID, TopAbs_SHELL, TopAbs_FACE

reader = IGESControl_Reader()
path2 = r"C:\Users\rathn\Downloads\new.IGS" 
path = r"C:\Users\rathn\Downloads\5e66b66e-a0d3-4a95-9824-e0d9914b0089.igs"
status = reader.ReadFile(path)
reader.TransferRoots()
shape = reader.OneShape()


In [89]:
status, shape.ShapeType()

(1, 0)

In [90]:
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.TopoDS import TopoDS_Face
from OCC.Core.TopoDS import TopoDS_Shape

# Create the TopExp_Explorer for iterating over faces in the shape
explorer = TopExp_Explorer(shape, TopAbs_FACE)
faces=[]
# Loop through all the faces in the shape
while explorer.More():
    current_shape = explorer.Current()
    if current_shape.ShapeType() == 4:
        faces.append(current_shape)
        explorer.Next()

In [91]:
faces[:5]

[<class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>]

In [ ]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import TopoDS_Edge, TopoDS_Vertex
import math

def are_faces_connected(face1, face2, tolerance=1e-5):
    """
    Check if two faces are connected by a common edge, vertex, or by proximity.
    A tolerance is used for proximity checks.
    """
    # Check for shared edges first (as before)
    explorer1 = TopExp_Explorer(face1, TopAbs_EDGE)
    explorer2 = TopExp_Explorer(face2, TopAbs_EDGE)

    while explorer1.More():
        edge1 = explorer1.Current()
        while explorer2.More():
            edge2 = explorer2.Current()
            # Check if the edges are exactly equal
            if edge1.IsEqual(edge2):
                return True
            explorer2.Next()
        explorer1.Next()

    # Check for shared vertices with a small tolerance
    explorer1 = TopExp_Explorer(face1, TopAbs_VERTEX)
    explorer2 = TopExp_Explorer(face2, TopAbs_VERTEX)
    
    while explorer1.More():
        vertex1 = TopoDS_Vertex(explorer1.Current())
        pt1 = BRep_Tool.Pnt(vertex1)
        while explorer2.More():
            vertex2 = TopoDS_Vertex(explorer2.Current())
            pt2 = BRep_Tool.Pnt(vertex2)
            # Check if the distance between vertices is smaller than the tolerance
            if pt1.Distance(pt2) < tolerance:
                return True
            explorer2.Next()
        explorer1.Next()

    # If no shared edges or sufficiently close vertices, faces are not connected
    return False


In [93]:
import networkx as nx

# Create a graph where each node represents a face
G = nx.Graph()

# Add nodes for each face
for i, face in enumerate(faces):
    G.add_node(i, face=face)

# Connect faces that share edges (you would need to check edge/vertex sharing)
for i, face1 in enumerate(faces):
    for j, face2 in enumerate(faces[i+1:], i+1):
        if are_faces_connected(face1, face2):  # You'll need to implement this
            G.add_edge(i, j)

# Find connected components (each component represents a part)
connected_components = list(nx.connected_components(G))

# Each connected component is a part
for part in connected_components:
    print(f"Part with faces: {part}")


Part with faces: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 668, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 673, 674, 651, 652, 653, 654, 655, 656, 658, 659, 660, 661, 662, 663, 664, 665, 157, 666, 667, 669, 670, 671, 672, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1015, 1016, 1017}
Part with faces: {74}
Part with faces: {75}
Part with faces: {76, 77, 80, 81, 82, 83, 84, 85, 86, 90, 91, 92}
Part with faces: {78, 79, 87, 89, 93}
Part with faces: {124, 119, 88, 121, 122, 123, 120, 125, 126}
Part with faces: {96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 1

In [94]:
len(faces),len(connected_components)

(1144, 101)

In [95]:
faces[:5]

[<class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>,
 <class 'TopoDS_Face'>]

In [96]:
from OCC.Core.BRep import BRep_Builder
from OCC.Core.TopoDS import TopoDS_Compound
from OCC.Extend.DataExchange import write_step_file

def export_components_to_step(connected_components, faces, prefix="component"):
    for i, component in enumerate(connected_components):
        compound = TopoDS_Compound()
        builder = BRep_Builder()
        builder.MakeCompound(compound)
        for idx in component:
            builder.Add(compound, faces[idx])
        
        write_step_file(compound, f"{prefix}_{i}.step")

# Call this:
export_components_to_step(connected_components, faces)


In [97]:
from OCC.Core.BRep import BRep_Builder
from OCC.Core.TopoDS import TopoDS_Compound
from OCC.Extend.DataExchange import write_stl_file

def export_components_to_stl(connected_components, faces, prefix="component"):
    for i, component in enumerate(connected_components):
        compound = TopoDS_Compound()
        builder = BRep_Builder()
        builder.MakeCompound(compound)
        for idx in component:
            builder.Add(compound, faces[idx])
        
        write_stl_file(compound, f"stl_compo\\{prefix}_{i}.stl")


In [98]:
export_components_to_stl(connected_components, faces)

In [99]:
len(connected_components),len(faces)

(101, 1144)

In [100]:
import open3d as o3d
import os
import numpy as np
pointclouds = []
filenames = []

stl_file_path = "C:\\Users\\rathn\\Downloads\\cad_project\\stl_compo"
for i in os.listdir(stl_file_path):
    filename = stl_file_path+"\\"+i
    mesh = o3d.io.read_triangle_mesh(filename)
    if not mesh.has_triangles():
        print(f"Skipping invalid mesh: {filename}")
    mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())

    pcd = mesh.sample_points_uniformly(number_of_points=1024)
    points = np.asarray(pcd.points)
    pointclouds.append(points)
    filenames.append(filename)


In [101]:
pointclouds = np.array(pointclouds)
pointclouds.shape

(101, 1024, 3)

In [102]:
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.StlAPI import StlAPI_Writer
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
from OCC.Core.TopoDS import topods

def convert_step_to_stl(step_file_path, stl_output_path, linear_deflection=0.01, angular_deflection=0.5):
    # Read STEP file
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(step_file_path)
    if status != 1:
        raise Exception("Error reading STEP file")

    step_reader.TransferRoots()
    shape = step_reader.OneShape()

    # Mesh the shape
    mesh = BRepMesh_IncrementalMesh(shape, linear_deflection, False, angular_deflection, True)
    mesh.Perform()

    # Export to STL
    stl_writer = StlAPI_Writer()
    stl_writer.Write(shape, stl_output_path)
    print(f"Saved: {stl_output_path}")


In [103]:
grndTruth = r"C:\Users\rathn\Downloads\abc_0000_step_v00\00009945\00009945_086229ea796640d29acd68ff_step_080.step"
outPath = r"C:\Users\rathn\Downloads\cad_project\out.stl"

In [104]:
convert_step_to_stl(grndTruth,outPath)

Saved: C:\Users\rathn\Downloads\cad_project\out.stl


In [105]:
import open3d as o3d
import os
import numpy as np


gt_file_path = "C:\\Users\\rathn\\Downloads\\cad_project\\out.stl"
mesh = o3d.io.read_triangle_mesh(gt_file_path)
if not mesh.has_triangles():
    print(f"Skipping invalid mesh: {filename}")
mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
pcd = mesh.sample_points_uniformly(number_of_points=1024)
gt_points = np.asarray(pcd.points)


In [110]:
import numpy as np
from scipy.spatial import KDTree

def suggest_tolerance_from_pointcloud(points, sample_size=100):
    """
    Suggest a tolerance value based on the average nearest neighbor distance
    from a sample of points in the point cloud.

    Parameters:
    - points: np.ndarray of shape (N, 3), your 3D point cloud
    - sample_size: number of points to sample for efficiency

    Returns:
    - suggested_tolerance: float
    """
    N = points.shape[0]
    sample_indices = np.random.choice(N, min(sample_size, N), replace=False)
    sampled_points = points[sample_indices]

    kdtree = KDTree(points)
    distances, _ = kdtree.query(sampled_points, k=2)  # k=2: nearest and self
    nearest_neighbor_dists = distances[:, 1]  # skip self-distance at index 0

    avg_dist = np.mean(nearest_neighbor_dists)
    min_dist = np.min(nearest_neighbor_dists)
    max_dist = np.max(nearest_neighbor_dists)

    suggested_tolerance = round(avg_dist * 1.5, 5)

    print(f"📏 Nearest neighbor distances (sample of {sample_size}):")
    print(f"  ↪ Avg: {avg_dist:.6f}, Min: {min_dist:.6f}, Max: {max_dist:.6f}")
    print(f"✅ Suggested Tolerance: {suggested_tolerance}")

    return suggested_tolerance


In [ ]:
# Assuming `points` is of shape (1024, 3)

suggested_tol = suggest_tolerance_from_pointcloud(pointclouds[2])


📏 Nearest neighbor distances (sample of 100):
  ↪ Avg: 0.031318, Min: 0.002464, Max: 0.069771
✅ Suggested Tolerance: 0.04698
